In [ ]:
## without PCA

import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, roc_auc_score, precision_recall_curve
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

# Directory paths
train_dir = '/kaggle/input/promise12-2d-dataset/PROMISE12/train_data'
val_dir = '/kaggle/input/promise12-2d-dataset/PROMISE12/validation_data'
test_dir = '/kaggle/input/promise12-2d-dataset/PROMISE12/test_data'

# Image dimensions
img_height = 256
img_width = 256
batch_size = 16

# Data augmentation for training images
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Data generators
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(val_dir,
                                                                       target_size=(img_height, img_width),
                                                                       batch_size=batch_size,
                                                                       class_mode='binary')

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(test_dir,
                                                                        target_size=(img_height, img_width),
                                                                        batch_size=batch_size,
                                                                        class_mode='binary',
                                                                        shuffle=False)

# Function to display sample images
def display_sample_images(directory, num_images=5):
    image_files = os.listdir(os.path.join(directory, 'image'))
    mask_files = os.listdir(os.path.join(directory, 'mask'))
    fig, axes = plt.subplots(num_images, 2, figsize=(10, 15))
    for i in range(num_images):
        image_path = os.path.join(directory, 'image', image_files[i])
        mask_path = os.path.join(directory, 'mask', mask_files[i])
        image = Image.open(image_path)
        mask = Image.open(mask_path)
        axes[i, 0].imshow(image)
        axes[i, 0].set_title('Image')
        axes[i, 0].axis('off')
        axes[i, 1].imshow(mask)
        axes[i, 1].set_title('Mask')
        axes[i, 1].axis('off')
    plt.show()

# Display sample images from train set
print("Sample images from train set:")
display_sample_images(train_dir)

# Display sample images from validation set
print("Sample images from validation set:")
display_sample_images(val_dir)

# Display sample images from test set
print("Sample images from test set:")
display_sample_images(test_dir)

# Data generators
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(val_dir,
                                                                       target_size=(img_height, img_width),
                                                                       batch_size=batch_size,
                                                                       class_mode='binary')

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(test_dir,
                                                                        target_size=(img_height, img_width),
                                                                        batch_size=batch_size,
                                                                        class_mode='binary',
                                                                        shuffle=False)

# Quadratic Discriminant Analysis (QDA) Model
# Load the training data
train_images = []
train_labels = []
for images, labels in train_generator:
    train_images.append(images)
    train_labels.append(labels)
    if len(train_images) * batch_size >= len(train_generator.filenames):
        break
train_images = np.concatenate(train_images)
train_labels = np.concatenate(train_labels)

# Load the validation data
val_images = []
val_labels = []
for images, labels in val_generator:
    val_images.append(images)
    val_labels.append(labels)
    if len(val_images) * batch_size >= len(val_generator.filenames):
        break
val_images = np.concatenate(val_images)
val_labels = np.concatenate(val_labels)

# Load the test data
test_images = []
test_labels = []
for images, labels in test_generator:
    test_images.append(images)
    test_labels.append(labels)
    if len(test_images) * batch_size >= len(test_generator.filenames):
        break
test_images = np.concatenate(test_images)
test_labels = np.concatenate(test_labels)

# Flatten the image data
train_images_flattened = train_images.reshape(train_images.shape[0], -1)
val_images_flattened = val_images.reshape(val_images.shape[0], -1)
test_images_flattened = test_images.reshape(test_images.shape[0], -1)

# Define and train the QDA model
qda_model = QuadraticDiscriminantAnalysis()
qda_model.fit(train_images_flattened, train_labels)

# Predictions on validation set
val_pred_qda = qda_model.predict(val_images_flattened)

# Metrics on validation set
val_accuracy_qda = accuracy_score(val_labels, val_pred_qda)
val_precision_qda = precision_score(val_labels, val_pred_qda)
val_auc_qda = roc_auc_score(val_labels, qda_model.predict_proba(val_images_flattened)[:,1])

print("Quadratic Discriminant Analysis Metrics on Validation Set:")
print(f'Validation Accuracy: {val_accuracy_qda}')
print(f'Validation Precision: {val_precision_qda}')
print(f'Validation AUC: {val_auc_qda}')

# Predictions on test set
test_pred_qda = qda_model.predict(test_images_flattened)

# Metrics on test set
test_accuracy_qda = accuracy_score(test_labels, test_pred_qda)
test_precision_qda = precision_score(test_labels, test_pred_qda)
test_auc_qda = roc_auc_score(test_labels, qda_model.predict_proba(test_images_flattened)[:,1])

print("Quadratic Discriminant Analysis Metrics on Test Set:")
print(f'Test Accuracy: {test_accuracy_qda}')
print(f'Test Precision: {test_precision_qda}')
print(f'Test AUC: {test_auc_qda}')

# Compute probabilities for the test set
test_probabilities_qda = qda_model.predict_proba(test_images_flattened)[:, 1]

# Compute ROC curve and AUC
fpr_qda, tpr_qda, thresholds_qda = roc_curve(test_labels, test_probabilities_qda)
roc_auc_qda = auc(fpr_qda, tpr_qda)

# Plot ROC curve
plt.figure()
plt.plot(fpr_qda, tpr_qda, color='darkorange', lw=2, label=f'AUC = {roc_auc_qda:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve (QDA)')
plt.legend(loc='lower right')
plt.show()

# Compute accuracy
accuracy_qda = accuracy_score(test_labels, test_pred_qda)
print(f'Accuracy (QDA): {accuracy_qda:.2f}')

# Compute precision
precision_qda = precision_score(test_labels, test_pred_qda)
print(f'Precision (QDA): {precision_qda:.2f}')

# Plot precision-recall curve
precision_qda, recall_qda, _ = precision_recall_curve(test_labels, test_probabilities_qda)
plt.figure()
plt.plot(recall_qda, precision_qda, color='blue', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (QDA)')
plt.show()


In [ ]:
##with PCA

import os
import numpy as np
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve

# Directory paths
train_dir = '/kaggle/input/promise12-2d-dataset/PROMISE12/train_data'
val_dir = '/kaggle/input/promise12-2d-dataset/PROMISE12/validation_data'
test_dir = '/kaggle/input/promise12-2d-dataset/PROMISE12/test_data'

# Image dimensions
img_height = 256
img_width = 256
batch_size = 16

# Data augmentation for training images
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Data generators
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(val_dir,
                                                                       target_size=(img_height, img_width),
                                                                       batch_size=1,  # Batch size set to 1
                                                                       class_mode='binary',
                                                                       shuffle=False)

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(test_dir,
                                                                        target_size=(img_height, img_width),
                                                                        batch_size=1,  # Batch size set to 1
                                                                        class_mode='binary',
                                                                        shuffle=False)

# Load the training data
train_images = []
train_labels = []
for images, labels in train_generator:
    train_images.append(images)
    train_labels.append(labels)
    if len(train_images) * batch_size >= len(train_generator.filenames):
        break
train_images = np.concatenate(train_images)
train_labels = np.concatenate(train_labels)

# Flatten the image data
train_images_flattened = train_images.reshape(train_images.shape[0], -1)

# Apply PCA for feature reduction
pca = PCA(n_components=50)  # Adjust the number of components as needed
train_images_reduced = pca.fit_transform(train_images_flattened)

# Define and train the QDA model
qda_model = QuadraticDiscriminantAnalysis(reg_param=0.1)  # Adjust regularization parameter as needed
qda_model.fit(train_images_reduced, train_labels)

# Predictions on validation set
val_images, val_labels = [], []
for _ in range(len(val_generator.filenames)):
    images, labels = next(val_generator)  # Corrected line to fetch next batch
    val_images.append(images)
    val_labels.append(labels)
val_images = np.concatenate(val_images)
val_labels = np.concatenate(val_labels)
val_images_flattened = val_images.reshape(val_images.shape[0], -1)
val_images_reduced = pca.transform(val_images_flattened)
val_pred_qda = qda_model.predict(val_images_reduced)

# Metrics on validation set
val_accuracy_qda = accuracy_score(val_labels, val_pred_qda)
val_precision_qda = precision_score(val_labels, val_pred_qda)
val_auc_qda = roc_auc_score(val_labels, qda_model.predict_proba(val_images_reduced)[:,1])

print("Quadratic Discriminant Analysis Metrics on Validation Set:")
print(f'Validation Accuracy: {val_accuracy_qda}')
print(f'Validation Precision: {val_precision_qda}')
print(f'Validation AUC: {val_auc_qda}')

# Predictions on test set
test_images, test_labels = [], []
for _ in range(len(test_generator.filenames)):
    images, labels = next(test_generator)  # Corrected line to fetch next batch
    test_images.append(images)
    test_labels.append(labels)
test_images = np.concatenate(test_images)
test_labels = np.concatenate(test_labels)
test_images_flattened = test_images.reshape(test_images.shape[0], -1)
test_images_reduced = pca.transform(test_images_flattened)
test_pred_qda = qda_model.predict(test_images_reduced)

# Metrics on test set
test_accuracy_qda = accuracy_score(test_labels, test_pred_qda)
test_precision_qda = precision_score(test_labels, test_pred_qda)
test_auc_qda = roc_auc_score(test_labels, qda_model.predict_proba(test_images_reduced)[:,1])

print("Quadratic Discriminant Analysis Metrics on Test Set:")
print(f'Test Accuracy: {test_accuracy_qda}')
print(f'Test Precision: {test_precision_qda}')
print(f'Test AUC: {test_auc_qda}')

# Compute probabilities for the test set
test_probabilities_qda = qda_model.predict_proba(test_images_reduced)[:, 1]

# Compute ROC curve and AUC
fpr_qda, tpr_qda, _ = roc_curve(test_labels, test_probabilities_qda)
roc_auc_qda = auc(fpr_qda, tpr_qda)

# Plot ROC curve
plt.figure()
plt.plot(fpr_qda, tpr_qda, color='darkorange', lw=2, label=f'AUC = {roc_auc_qda:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve (QDA)')
plt.legend(loc='lower right')
plt.show()

# Compute precision-recall curve
precision_qda, recall_qda, _ = precision_recall_curve(test_labels, test_probabilities_qda)

# Plot precision-recall curve
plt.figure()
plt.plot(recall_qda, precision_qda, color='blue', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (QDA)')
plt.show()
